# Notebook configuration

In [273]:
# Load project config
import toml

config = toml.load('../config.toml')
print(config)

{'title': 'Sprint 3b config', 'input_data': {'shapefile_dir_path': '/home/william/Programming/WazeTT/data/input/bd_topo/ZONE_VEGETATION/', 'shapefile_name': 'ZONE_VEGETATION.shp', 'ascii_file_dir_path': '../data/input/bd_alti/', 'ascii_file_name': 'BDALTIV2_25M_FXX_0825_6550_MNT_LAMB93_IGN69.asc'}, 'nominal_speeds': {'walk': {'reference': 100, 'bush': 0.3, 'sand': 0.6, 'grass': 1, 'road': 1, 'building': 0, 'water': 0, 'snow': 0.8, 'angle': [{'range': [15, 90], 'coef': 0}, {'range': [10, 15], 'coef': 0.2}, {'range': [5, 10], 'coef': 0.6}, {'range': [0, 5], 'coef': 1}, {'range': [-5, 0], 'coef': 1.1}, {'range': [-15, -5], 'coef': 1.3}, {'range': [-90, -15], 'coef': 0}]}}, 'isochrones': [{'id': 1, 'time': 5}, {'id': 2, 'time': 10}, {'id': 3, 'time': 30}]}


In [274]:
# Config import examples
shapefile_path = config['input_data']['shapefile_dir_path'] + config['input_data']['shapefile_name']
print(shapefile_path)

ascii_file_path = config['input_data']['ascii_file_dir_path'] + config['input_data']['ascii_file_name']
print(ascii_file_path)

isochrones = config['isochrones']
for isochrone in isochrones:
    print(isochrones)

nominal_speed_walk = config['nominal_speeds']['walk']
print()
print(nominal_speed_walk)
for angle in config['nominal_speeds']['walk']['angle']:
    for range, coef in angle.items():
        print('{}\t{}'.format(range, coef))

/home/william/Programming/WazeTT/data/input/bd_topo/ZONE_VEGETATION/ZONE_VEGETATION.shp
../data/input/bd_alti/BDALTIV2_25M_FXX_0825_6550_MNT_LAMB93_IGN69.asc
[{'id': 1, 'time': 5}, {'id': 2, 'time': 10}, {'id': 3, 'time': 30}]
[{'id': 1, 'time': 5}, {'id': 2, 'time': 10}, {'id': 3, 'time': 30}]
[{'id': 1, 'time': 5}, {'id': 2, 'time': 10}, {'id': 3, 'time': 30}]

{'reference': 100, 'bush': 0.3, 'sand': 0.6, 'grass': 1, 'road': 1, 'building': 0, 'water': 0, 'snow': 0.8, 'angle': [{'range': [15, 90], 'coef': 0}, {'range': [10, 15], 'coef': 0.2}, {'range': [5, 10], 'coef': 0.6}, {'range': [0, 5], 'coef': 1}, {'range': [-5, 0], 'coef': 1.1}, {'range': [-15, -5], 'coef': 1.3}, {'range': [-90, -15], 'coef': 0}]}
range	[15, 90]
coef	0
range	[10, 15]
coef	0.2
range	[5, 10]
coef	0.6
range	[0, 5]
coef	1
range	[-5, 0]
coef	1.1
range	[-15, -5]
coef	1.3
range	[-90, -15]
coef	0


# Load 2D tables

In [275]:
# from alti_data import AltiData
# alti_data = AltiData(ascii_file_path)
import numpy as np

# alti_table = np.full((50,50), 10)
alti_table = np.array([ [20,20,19,19,16],
                        [20,20,22,17,20],
                        [18,19,20,18,13],
                        [19,20,22,20,20],
                        [18,21,21,21,25]])
field_table = np.array([    ['water','road','snow','road','road'],
                            ['road','water','snow','road','road'],
                            ['sand','grass','grass','bush','bush'],
                            ['sand','sand','grass','grass','bush'],
                            ['grass','sand','grass','bush','bush']])

# print(alti_table)
print(field_table[2,2])


grass


# Calculate departure coordinates

In [276]:
# departure_coord = (
#     alti_table['xllcorner_lamb93']+(alti_table ['ncols']*alti_table['cellsize'])/2,
#     alti_table['yllcorner_lamb93']+(alti_table ['nrows']*alti_table['cellsize'])/2
# )
# print(departure_coord)

# Calculate isochrone(s)

### Configuration variables

In [277]:
import numpy as np
from math import sqrt, acos, degrees

speed = config['nominal_speeds']['walk']['reference']
field_coef = config['nominal_speeds']['walk']
angle_coef = config['nominal_speeds']['walk']['angle']
# alpha = alpha[0]['coef']

print (field_coef)


{'reference': 100, 'bush': 0.3, 'sand': 0.6, 'grass': 1, 'road': 1, 'building': 0, 'water': 0, 'snow': 0.8, 'angle': [{'range': [15, 90], 'coef': 0}, {'range': [10, 15], 'coef': 0.2}, {'range': [5, 10], 'coef': 0.6}, {'range': [0, 5], 'coef': 1}, {'range': [-5, 0], 'coef': 1.1}, {'range': [-15, -5], 'coef': 1.3}, {'range': [-90, -15], 'coef': 0}]}


### Cost Function

In [278]:


# inputs : indice initial, primary move, alti_table, field_table
# output : delta

# Taille de cellule

dalle_length = 25 # m

initial_position = (2, 2)

primary_move = [(0,1),(1,1),(1,0),(1,-1),(0,-1),(-1,-1),(-1,0),(-1,1)]

def cost(initial_summit: tuple, primary_move: tuple, alti_table: list, field_table: list) -> float:

    denivele_initial = alti_table [initial_summit[0]][initial_summit[1]]

    #Diagonal movement or lateral movement
    if primary_move == (1,1) or primary_move == (1,-1) or primary_move == (-1,1) or primary_move == (-1,-1):
        distance = dalle_length*sqrt(2)
    else:
        distance = dalle_length
    denivele = alti_table [initial_summit[0]+primary_move[0]][initial_summit[1]+primary_move[1]]-denivele_initial
    if denivele < 0:
        angle = -degrees(acos(distance/sqrt(distance*distance+denivele*denivele)))
    else :
        angle = degrees(acos(distance/sqrt(distance*distance+denivele*denivele)))

    
    #Determining the coefficient alpha linked to the field
    milieu1 = field_table[initial_summit[0]][initial_summit[1]]
    milieu2 = field_table[initial_summit[0]+primary_move[0]][initial_summit[1]+primary_move[1]]

    alpha1 = field_coef[milieu1]
    alpha2 = field_coef[milieu2]

    #Determining the coefficient beta linked to the angle
    if angle_coef[0]['range'][0] <= angle <= angle_coef[0]['range'][1]:
        beta = angle_coef[0]['coef']
    elif angle_coef[1]['range'][0] <= angle < angle_coef[1]['range'][1]:
        beta = angle_coef[1]['coef']
    elif angle_coef[2]['range'][0] <= angle < angle_coef[2]['range'][1]:
        beta = angle_coef[2]['coef']
    elif angle_coef[3]['range'][0] <= angle < angle_coef[3]['range'][1]:
        beta = angle_coef[3]['coef']
    elif angle_coef[4]['range'][0] <= angle < angle_coef[4]['range'][1]:
        beta = angle_coef[4]['coef']
    elif angle_coef[5]['range'][0] <= angle < angle_coef[5]['range'][1]:
        beta = angle_coef[5]['coef']
    else:
        beta = angle_coef[6]['coef']

    
    #Calculate the time to go from the initial position to the target position
    if beta == 0 or milieu1 == 'water' or milieu2 == 'water':
        return -100
    else :
        if primary_move == (1,1) or primary_move == (1,-1) or primary_move == (-1,1) or primary_move == (-1,-1):
            return (sqrt(distance*distance+denivele*denivele))/(((alpha1+alpha2)/2)*beta*speed)
        else:
            return (sqrt(dalle_length*dalle_length+denivele*denivele))/(((alpha1+alpha2)/2)*beta*speed)



### Graph initialization

In [279]:
# initial_summit = neighbors[0]

unknown_table = np.full(alti_table.shape, 100, dtype =float)
print(unknown_table)


[[100. 100. 100. 100. 100.]
 [100. 100. 100. 100. 100.]
 [100. 100. 100. 100. 100.]
 [100. 100. 100. 100. 100.]
 [100. 100. 100. 100. 100.]]


### Graph Algorithm

In [280]:
initial_time = 10
unknown_table[initial_position] = initial_time
ancient_neighbors = []

#Initialization of the 8 neighbors
neighbors = []
neighbors=[initial_position]


#### Testing cost function for one movement

In [281]:
print (unknown_table[initial_position])
mv = (-1,1)
test = unknown_table[initial_position] - cost(initial_position, mv, alti_table, field_table)
print (test)
# print (cost(initial_position, mv, alti_table, field_table))
# print(-degrees(acos(dalle_lenght*sqrt(2)/sqrt(dalle_lenght*sqrt(2)*dalle_lenght*sqrt(2)+4*4))))
# print(angle_coef[5]['range'][0])
# print(angle_coef[5]['range'][1])
# print(angle_coef[5]['coef'])

10.0
9.677432818196179


In [282]:
import time
from IPython.display import display, clear_output

while len(neighbors) > 0 :
    
    #  1-Take an initial position on the graph
    #  2-Choose the first summit in the stack
    #  3-Annotate the neighbors of this summit
    #    a) For each neighbor of this summit : 
    #      - It is part of the boundary = > nothing to do
    #      - It is unknown => calculate the movement; add it to the boundary, remove from the boundary list
    
    initial_summit = neighbors[0]
    for i in primary_move :
        if (0 <= initial_summit[0]+i[0] < unknown_table.shape[0]) and (0 <= initial_summit[1]+i[1] < unknown_table.shape[1]) :
            # Check whether the target summit does not go out of the map
            target_summit=((initial_summit[0]+i[0],initial_summit[1]+i[1]))
            time_cost = cost(initial_summit, i, alti_table, field_table)
            if unknown_table[target_summit] == 100 or \
            initial_time > unknown_table[initial_summit] - time_cost > unknown_table[target_summit]: # unknown summit
                if unknown_table[initial_summit] - time_cost >= 0 : # possible move
                    unknown_table[target_summit] = unknown_table[initial_summit] - time_cost
                if target_summit not in neighbors and field_table[target_summit] != 'water':
                    neighbors.append(target_summit)
                    ancient_neighbors.append(target_summit)

    if initial_summit in neighbors : 
        neighbors.remove(initial_summit)
    
    clear_output(wait=True)
    display(np.around(unknown_table,decimals=3))
    display(initial_summit)
    time.sleep(1)

print("Graph skimmed")

array([[109.419,   9.364,   9.479,   9.427,   9.356],
       [  9.419, 110.   ,   9.721,   9.677,   9.258],
       [  9.488,   9.773,  10.   ,   9.649,   9.256],
       [  9.331,   9.558,   9.749,   9.646,   9.262],
       [  9.156,   9.347,   9.522,   9.262,   8.731]])

(0, 4)

Graph skimmed


In [272]:
print(np.around(unknown_table,decimals=3))

[[109.419   9.364   9.479   9.427   9.356]
 [  9.419 110.      9.721   9.677   9.258]
 [  9.488   9.773  10.      9.649   9.256]
 [  9.331   9.558   9.749   9.646   9.262]
 [  9.156   9.347   9.522   9.262   8.731]]


# Plot isochrone(s) on map